# DataFrames (Expert / Advanced++)

This notebook is **expert-level** practice on pandas DataFrames.

## What makes these problems Advanced++
- Time-aware joins and analytics with timestamps
- Group-level computations with `groupby`, `transform`, and `NamedAgg`
- Reshaping data with pivoting / stacking / tidy formats
- Robust handling of missing values and validation checks
- Performance-aware, vectorized solutions (no unnecessary Python loops)

Each problem includes:
- a **Problem** cell (try it first)
- a **Solution** cell

> Best practice: attempt the problem first, then compare with the solution.

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)

rng = np.random.default_rng(7)

## Shared synthetic dataset: E-commerce events

We'll generate:
- `orders`: order-level data
- `events`: user session events
- `rates`: FX rates over time (for `merge_asof`)

The data is small, but structured like real-world analytics.

In [2]:
# --- Orders ---
n_orders = 600
order_ids = np.arange(10_000, 10_000 + n_orders)

users = rng.integers(1000, 1100, size=n_orders)
countries = rng.choice(["US", "CA", "GB", "DE"], size=n_orders, p=[0.45, 0.15, 0.2, 0.2])
currency_map = {"US": "USD", "CA": "CAD", "GB": "GBP", "DE": "EUR"}
currency = pd.Series(countries).map(currency_map).to_numpy()

start = pd.Timestamp("2025-01-01")
order_time = start + pd.to_timedelta(rng.integers(0, 60 * 24 * 30, size=n_orders), unit="m")  # ~30 days

category = rng.choice(["electronics", "home", "fashion", "books"], size=n_orders, p=[0.25, 0.25, 0.35, 0.15])
qty = rng.integers(1, 6, size=n_orders)

# price per item, different distributions per category
price = np.where(
    category == "electronics", rng.normal(220, 80, size=n_orders),
    np.where(category == "home", rng.normal(70, 25, size=n_orders),
             np.where(category == "fashion", rng.normal(55, 30, size=n_orders),
                      rng.normal(18, 8, size=n_orders)))
)
price = np.clip(price, 3, None)

discount = rng.choice([0.0, 0.05, 0.10, 0.15, 0.25], size=n_orders, p=[0.55, 0.15, 0.15, 0.1, 0.05])

orders = pd.DataFrame({
    "order_id": order_ids,
    "user_id": users,
    "country": countries,
    "currency": currency,
    "order_time": order_time,
    "category": category,
    "qty": qty,
    "unit_price": price.round(2),
    "discount": discount,
}).sort_values("order_time").reset_index(drop=True)

orders["gross_local"] = orders["qty"] * orders["unit_price"]
orders["net_local"] = orders["gross_local"] * (1 - orders["discount"])

orders.head()

,order_id,user_id,country,currency,order_time,category,qty,unit_price,discount,gross_local,net_local
0,10596,1008,GB,GBP,2025-01-01 02:47:00,electronics,4,260.40,0.10,1041.60,937.4400
1,10393,1002,US,USD,2025-01-01 04:14:00,electronics,2,227.67,0.00,455.34,455.3400
2,10047,1004,DE,EUR,2025-01-01 05:52:00,fashion,5,34.03,0.15,170.15,144.6275
3,10270,1021,DE,EUR,2025-01-01 08:15:00,electronics,3,166.09,0.00,498.27,498.2700
4,10471,1002,DE,EUR,2025-01-01 09:09:00,home,3,131.91,0.10,395.73,356.1570


In [3]:
# --- Session events ---
n_events = 2200
session_ids = rng.integers(50_000, 55_000, size=n_events)
event_user = rng.integers(1000, 1100, size=n_events)
event_time = start + pd.to_timedelta(rng.integers(0, 60 * 24 * 30, size=n_events), unit="m")

event_type = rng.choice(["view", "add_to_cart", "checkout"], size=n_events, p=[0.78, 0.17, 0.05])
channel = rng.choice(["organic", "ads", "email", "social"], size=n_events, p=[0.45, 0.25, 0.2, 0.1])

events = pd.DataFrame({
    "session_id": session_ids,
    "user_id": event_user,
    "event_time": event_time,
    "event_type": event_type,
    "channel": channel,
}).sort_values(["event_time", "user_id"]).reset_index(drop=True)

events.head()

,session_id,user_id,event_time,event_type,channel
0,52432,1078,2025-01-01 00:05:00,view,organic
1,50584,1028,2025-01-01 00:21:00,view,email
2,52163,1032,2025-01-01 00:22:00,add_to_cart,organic
3,52112,1041,2025-01-01 00:23:00,view,email
4,53657,1086,2025-01-01 01:07:00,view,organic


In [4]:
# --- FX rates (to USD) ---
rate_times = pd.date_range(start, periods=4 * 30, freq="6H")

rates = []
for cur in ["USD", "EUR", "GBP", "CAD"]:
    base = {"USD": 1.0, "EUR": 1.08, "GBP": 1.27, "CAD": 0.74}[cur]
    noise = rng.normal(0, 0.01, size=len(rate_times))
    series = (base + noise).clip(0.5, 2.0)
    rates.append(pd.DataFrame({"rate_time": rate_times, "currency": cur, "usd_rate": series}))

rates = pd.concat(rates, ignore_index=True).sort_values(["rate_time", "currency"]).reset_index(drop=True)
rates.head()

C:\Users\user1\AppData\Local\Temp\ipykernel_1892\1477531767.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rate_times = pd.date_range(start, periods=4 * 30, freq="6H")


,rate_time,currency,usd_rate
0,2025-01-01 00:00:00,CAD,0.745863
1,2025-01-01 00:00:00,EUR,1.087045
2,2025-01-01 00:00:00,GBP,1.275159
3,2025-01-01 00:00:00,USD,1.017489
4,2025-01-01 06:00:00,CAD,0.727239


# Problem 1 — Time-aware join with `merge_asof` (FX conversion)

### Task
Convert each order's `net_local` into USD as `net_usd`, using the **most recent FX rate at or before `order_time`**.

Requirements:
- Use `pd.merge_asof`.
- Match by `currency`.
- Output: `orders_fx` with all original columns plus `usd_rate` and `net_usd`.
- Validate: `usd_rate` has **no missing values**.


In [5]:
# STARTER
orders_fx = None
orders_fx

In [6]:
# SOLUTION (FIXED)
# IMPORTANT: merge_asof requires left/right 'on' keys to be globally sorted.
# Therefore we sort by [on, by], not [by, on].

o = orders.sort_values(["order_time", "currency"]).reset_index(drop=True)
r = rates.sort_values(["rate_time", "currency"]).reset_index(drop=True)

orders_fx = pd.merge_asof(
    o,
    r,
    left_on="order_time",
    right_on="rate_time",
    by="currency",
    direction="backward",
)

orders_fx["net_usd"] = orders_fx["net_local"] * orders_fx["usd_rate"]

assert orders_fx["usd_rate"].notna().all()
orders_fx.head()

,order_id,user_id,country,currency,order_time,category,qty,unit_price,discount,gross_local,net_local,rate_time,usd_rate,net_usd
0,10596,1008,GB,GBP,2025-01-01 02:47:00,electronics,4,260.40,0.10,1041.60,937.4400,2025-01-01 00:00:00,1.275159,1195.385420
1,10393,1002,US,USD,2025-01-01 04:14:00,electronics,2,227.67,0.00,455.34,455.3400,2025-01-01 00:00:00,1.017489,463.303602
2,10047,1004,DE,EUR,2025-01-01 05:52:00,fashion,5,34.03,0.15,170.15,144.6275,2025-01-01 00:00:00,1.087045,157.216604
3,10270,1021,DE,EUR,2025-01-01 08:15:00,electronics,3,166.09,0.00,498.27,498.2700,2025-01-01 06:00:00,1.069056,532.678420
4,10471,1002,DE,EUR,2025-01-01 09:09:00,home,3,131.91,0.10,395.73,356.1570,2025-01-01 06:00:00,1.069056,380.751697


# Problem 2 — Session funnel per channel using pivoting

### Task
Compute a funnel table per `channel` with counts of **unique sessions** that had:
- at least one `view`
- at least one `add_to_cart`
- at least one `checkout`

Output format:
- index: `channel`
- columns: `view`, `add_to_cart`, `checkout`
- values: number of unique `session_id`

Store result as `funnel`.

Constraint:
- Avoid loops.


In [7]:
# STARTER
funnel = None
funnel

In [8]:
# SOLUTION
funnel = (
    events.loc[:, ["channel", "event_type", "session_id"]]
    .drop_duplicates()
    .groupby(["channel", "event_type"], as_index=True)["session_id"]
    .nunique()
    .unstack("event_type", fill_value=0)
    .reindex(columns=["view", "add_to_cart", "checkout"], fill_value=0)
    .sort_index()
)

assert set(funnel.columns) == {"view", "add_to_cart", "checkout"}
funnel

event_type,view,add_to_cart,checkout
channel,,,
ads,386,107,35
email,324,96,15
organic,716,174,38
social,170,36,9


# Problem 3 — User-level attribution: last-touch channel before each purchase

Attribute each order to the **most recent event channel** for that user **strictly before** the order time.

### Task
Create `orders_attr` with columns:
- `order_id`, `user_id`, `order_time`, `net_local`
- `last_channel`
- `minutes_since_last_event`

Rules:
- Use `merge_asof` grouped by `user_id`.
- Only consider events with `event_time < order_time`.
- If no prior event: `last_channel='unknown'`, minutes = `NaN`.


In [9]:
# STARTER
orders_attr = None
orders_attr

In [10]:
# SOLUTION (FIXED)
# Sort by [on, by] where on=order_time_eps/event_time and by=user_id

o = orders[["order_id", "user_id", "order_time", "net_local"]].copy()
o["order_time_eps"] = o["order_time"] - pd.Timedelta("1ns")  # strict less-than
o = o.sort_values(["order_time_eps", "user_id"]).reset_index(drop=True)

e = events[["user_id", "event_time", "channel"]].sort_values(["event_time", "user_id"]).reset_index(drop=True)

joined = pd.merge_asof(
    o,
    e,
    left_on="order_time_eps",
    right_on="event_time",
    by="user_id",
    direction="backward",
)

orders_attr = joined.rename(columns={"channel": "last_channel"})
orders_attr["minutes_since_last_event"] = (orders_attr["order_time"] - orders_attr["event_time"]).dt.total_seconds() / 60

orders_attr["last_channel"] = orders_attr["last_channel"].fillna("unknown")
orders_attr = orders_attr.drop(columns=["order_time_eps", "event_time"])  # keep requested schema

assert orders_attr["last_channel"].notna().all()
orders_attr.head()

,order_id,user_id,order_time,net_local,last_channel,minutes_since_last_event
0,10596,1008,2025-01-01 02:47:00,937.4400,unknown,NaN
1,10393,1002,2025-01-01 04:14:00,455.3400,unknown,NaN
2,10047,1004,2025-01-01 05:52:00,144.6275,unknown,NaN
3,10270,1021,2025-01-01 08:15:00,498.2700,unknown,NaN
4,10471,1002,2025-01-01 09:09:00,356.1570,unknown,NaN


# Problem 4 — Cohorts: first purchase week + retention matrix

### Task
Build a **weekly cohort retention** table from `orders`:
- cohort = each user's **first purchase week**
- compute unique users purchasing at week_0, week_1, ... relative to cohort

Output:
- index: `cohort_week` (Timestamp)
- columns: `week_0`, `week_1`, ...
- values: unique user counts

Store in `retention`.

In [11]:
# STARTER
retention = None
retention

In [12]:
# SOLUTION
tmp = orders[["user_id", "order_time"]].copy()
tmp["order_week"] = tmp["order_time"].dt.to_period("W").dt.start_time
tmp["cohort_week"] = tmp.groupby("user_id")["order_week"].transform("min")
tmp["week_index"] = ((tmp["order_week"] - tmp["cohort_week"]) / np.timedelta64(1, "W")).astype(int)

counts = (
    tmp.groupby(["cohort_week", "week_index"])["user_id"]
       .nunique()
       .rename("users")
       .reset_index()
)

retention = (
    counts.pivot(index="cohort_week", columns="week_index", values="users")
          .fillna(0)
          .astype(int)
          .sort_index()
)
retention.columns = [f"week_{i}" for i in retention.columns]

assert (retention >= 0).all().all()
retention

,week_0,week_1,week_2,week_3,week_4
cohort_week,,,,,
2024-12-30,64,44,58,48,40
2025-01-06,34,25,26,25,0
2025-01-20,2,0,0,0,0


# Problem 5 — Percent-of-total within groups using `transform`

### Task
Compute each order's share of total `net_usd` **within its (country, category)** group.

Output:
- DataFrame `share_df` with:
  `order_id`, `country`, `category`, `net_usd`, `group_share`

Constraints:
- Use `groupby(...).transform('sum')`.
- Validate: within each group, shares sum to ~1.

Note: This solution recomputes `orders_fx` safely (so execution order doesn't matter).

In [13]:
# STARTER
share_df = None
share_df

In [14]:
# SOLUTION (ROBUST)
# Recompute orders_fx using the FIXED sorting rule.

o = orders.sort_values(["order_time", "currency"]).reset_index(drop=True)
r = rates.sort_values(["rate_time", "currency"]).reset_index(drop=True)

orders_fx_local = pd.merge_asof(
    o,
    r,
    left_on="order_time",
    right_on="rate_time",
    by="currency",
    direction="backward",
)
orders_fx_local["net_usd"] = orders_fx_local["net_local"] * orders_fx_local["usd_rate"]
assert orders_fx_local["usd_rate"].notna().all()

group_sum = orders_fx_local.groupby(["country", "category"])["net_usd"].transform("sum")

share_df = orders_fx_local.assign(group_share=orders_fx_local["net_usd"] / group_sum)
share_df = share_df.loc[:, ["order_id", "country", "category", "net_usd", "group_share"]]

check = share_df.groupby(["country", "category"])["group_share"].sum()
assert np.allclose(check.values, np.ones(len(check)), rtol=1e-8, atol=1e-8)

share_df.head()

,order_id,country,category,net_usd,group_share
0,10596,GB,electronics,1195.385420,0.062150
1,10393,US,electronics,463.303602,0.010400
2,10047,DE,fashion,157.216604,0.024264
3,10270,DE,electronics,532.678420,0.030579
4,10471,DE,home,380.751697,0.072865


# Problem 6 — Reshape: wide-to-long and long-to-wide with strict schema

### Task
Create a wide table of metrics by country & category:
- index: `country`
- columns: MultiIndex (`metric`, `category`) where metric in {`orders`, `net_local_sum`}

Then convert to tidy long and back, validating round-trip.

Store:
- `wide_metrics`
- `tidy_metrics`
- `wide_back`

In [15]:
# STARTER
wide_metrics = None
tidy_metrics = None
wide_back = None

wide_metrics

In [16]:
# SOLUTION
agg = orders.groupby(["country", "category"]).agg(
    orders=("order_id", "size"),
    net_local_sum=("net_local", "sum"),
)

wide_metrics = agg.unstack("category")

tidy_metrics = (
    wide_metrics
    .stack(level=1)  # category
    .rename_axis(index=["country", "category"])
    .reset_index()
    .melt(id_vars=["country", "category"], var_name="metric", value_name="value")
    .loc[:, ["country", "metric", "category", "value"]]
    .sort_values(["country", "metric", "category"], ignore_index=True)
)

wide_back = (
    tidy_metrics
    .pivot_table(index="country", columns=["metric", "category"], values="value", aggfunc="first")
    .sort_index(axis=1)
)

wide_metrics_sorted = wide_metrics.sort_index(axis=1)
assert wide_back.shape == wide_metrics_sorted.shape
assert np.allclose(wide_back.to_numpy(dtype=float), wide_metrics_sorted.to_numpy(dtype=float), rtol=1e-10, atol=1e-10)

wide_metrics

C:\Users\user1\AppData\Local\Temp\ipykernel_1892\3686415616.py:11: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(level=1)  # category


orders                          net_local_sum                                    
category  books electronics fashion home         books electronics     fashion        home
country                                                                                   
CA            7          24      30   28      354.5050  16489.8065   4556.9165   5122.6375
DE           14          23      42   27     1014.4700  16072.4000   5980.4785   4851.4170
GB           21          27      36   40      922.6945  15144.7235   5837.8930   7653.2185
US           44          73      87   77     2188.6960  44457.7305  16063.8565  14161.5615

# Problem 7 — Outlier detection per group with robust z-scores

### Task
Flag outliers within each category using robust z-scores:
- center: median
- scale: MAD
- robust_z = 0.6745 * (x - median) / MAD

Store:
- full annotated DataFrame: `orders_robust`
- outliers-only DataFrame: `outliers`


In [17]:
# STARTER
orders_robust = None
outliers = None

outliers

In [18]:
# SOLUTION
x = orders["net_local"]
grp = orders.groupby("category")
med = grp["net_local"].transform("median")
mad = grp["net_local"].transform(lambda s: np.median(np.abs(s - np.median(s))))

robust_z = 0.6745 * (x - med) / mad.replace(0, np.nan)

orders_robust = orders.assign(robust_z=robust_z)
orders_robust["is_outlier"] = orders_robust["robust_z"].abs() >= 3.5

outliers = (
    orders_robust.loc[orders_robust["is_outlier"], ["order_id", "category", "net_local", "robust_z"]]
    .assign(abs_z=lambda d: d["robust_z"].abs())
    .sort_values("abs_z", ascending=False)
    .drop(columns=["abs_z"])
    .reset_index(drop=True)
)

outliers.head()

,order_id,category,net_local,robust_z
0,10184,books,181.15,4.438770
1,10229,home,544.10,4.036269
2,10236,home,529.55,3.882895
3,10257,fashion,567.65,3.774067
4,10079,home,518.85,3.770104


# Problem 8 — Integrity constraints & reconciliation checks

### Task
Write `validate_orders(df)` that raises `AssertionError` if:
1. `order_id` not unique
2. `qty` not integer-like or < 1
3. `discount` not in [0,1)
4. `gross_local != qty * unit_price` (tol 1e-6)
5. `net_local != gross_local * (1-discount)` (tol 1e-6)

Then call it on `orders`.


In [19]:
# STARTER
def validate_orders(df: pd.DataFrame) -> None:
    ...

validate_orders(orders)

In [20]:
# SOLUTION
def validate_orders(df: pd.DataFrame) -> None:
    required = {"order_id", "qty", "unit_price", "discount", "gross_local", "net_local"}
    missing = required - set(df.columns)
    assert not missing, f"Missing columns: {missing}"

    assert df["order_id"].is_unique, "order_id must be unique"

    qty = df["qty"]
    assert pd.api.types.is_integer_dtype(qty) or np.all(np.equal(qty, np.floor(qty))), "qty must be integer-like"
    assert (qty >= 1).all(), "qty must be >= 1"

    disc = df["discount"]
    assert (disc >= 0).all() and (disc < 1).all(), "discount must be in [0, 1)"

    gross_expected = df["qty"] * df["unit_price"]
    assert np.allclose(df["gross_local"].to_numpy(float), gross_expected.to_numpy(float), rtol=0, atol=1e-6), "gross_local mismatch"

    net_expected = df["gross_local"] * (1 - df["discount"])
    assert np.allclose(df["net_local"].to_numpy(float), net_expected.to_numpy(float), rtol=0, atol=1e-6), "net_local mismatch"


validate_orders(orders)
"Validation passed"

'Validation passed'